In [50]:
import pandas as pd
import os
from tqdm import tqdm
import pydicom
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision.transforms as T
import segmentation_models_pytorch as smp
from torch.cuda.amp import autocast
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
from utils import *
from torcheval.metrics import R2Score
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
df = pd.read_csv('cleaned_data.csv')
df = df.dropna(subset='Measurement at Wrist')
mean = df['Age'].mean()
df['Age'].fillna(mean, inplace=True)

/tmp/ipykernel_962621/246703226.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(mean, inplace=True)


In [3]:
df.head()

,Number,WristImage Segmentation for AI Analysis,Arm (L/R),Sex,Race,Ethnicity,Age,Clinical Signs of CTS,CTS-6 Score,Measurement at Wrist
0,21,Yes,R,M,White,Non-Hispanic,43.0,Y,12.5,12.16
1,22,Yes,R,M,White,Unreported/Chose not to disclose,74.0,Y,16.5,20.10
2,23,Yes,L,M,White,Unreported/Chose not to disclose,74.0,Y,11.5,13.83
3,24,Yes,R,F,Black,Non-Hispanic,63.0,Y,21.0,16.55
4,25,Yes,R,M,White,Non-Hispanic,49.0,Y,9.0,19.21


In [4]:
df.isna().sum()

Number                                      0
WristImage Segmentation for AI Analysis     0
Arm (L/R)                                   0
Sex                                         0
Race                                        0
Ethnicity                                   0
Age                                         0
Clinical Signs of CTS                       0
CTS-6 Score                                16
Measurement at Wrist                        0
dtype: int64

In [5]:
test_df = df.dropna(subset=['Clinical Signs of CTS', 'CTS-6 Score'])

In [9]:
restructured_root = 'data_yolo'
image_dir = os.path.join(restructured_root, 'images')
image_train_dir = os.path.join(image_dir, 'train')
image_val_dir = os.path.join(image_dir, 'val')
image_test_dir = os.path.join(image_dir, 'test')
label_dir = os.path.join(restructured_root, 'labels')
label_train_dir = os.path.join(label_dir, 'train')
label_val_dir = os.path.join(label_dir, 'val')

In [7]:
anno_dir = '/data_vault/hexai02/CarpalTunnel/Annotations'
dicom_dir = '/data_vault/hexai02/CarpalTunnel/Images'

In [11]:
from Utilities.utilities import load_dicom, load_mask

In [12]:
model = YOLO("runs/detect/train17/weights/best.pt")
model.eval()
print('YOLO loaded')

YOLO loaded


In [13]:
config = {
    'epochs': 40,
    'in_channels': 1,
    'dropout': 0.2,
    'decoder_attention_type': 'scse',
    'init_lr': 5e-4,
    'weight_decay': 0.05,
    'T_max': 10,
    'eta_min': 3e-5
}

In [32]:

from Utilities.utilities import load_model, save_model

In [15]:
device = "cuda" if torch.cuda.is_available() else "cpu"

unet_model = smp.Unet(encoder_name="resnet18",in_channels=1,dropout=config['in_channels'],decoder_attention_type=config['decoder_attention_type']).to(device)

# unet_model, _, _, _ = load_model(unet_model, optimizer=None, scheduler=None, path=f'unet_runs/epoch_40.pth')
checkpoint = torch.load('unet_runs/epoch_40.pth', map_location="cuda")
unet_model.load_state_dict(checkpoint, strict=True)
unet_model.eval()
print('UNET loaded')

UNET loaded


In [18]:
classModel = FullModel(ConvNext(in_chans=2,dims=[96,192,384,768],stages=[1,1,3,1]), 
                  ClassificationTail(768,num_classes=2,dropout_rate=0.2)).to(device)

In [19]:
checkpoint_path = 'classification_runs/epoch_7.pth'
state_dict = torch.load(checkpoint_path, map_location='cuda')  # or 'cuda' if using GPU
classModel.load_state_dict(state_dict)

classHead = classModel.backbone

In [23]:
train_images = []
train_labels = []
val_images = []
val_labels = []


image_transform = T.Compose([
    T.Resize((512, 512), interpolation=T.InterpolationMode.BILINEAR),  
    T.Normalize(mean=[0.5], std=[0.5]),  
])

def fill_arr(img_dir, img_bucket, mask_bucket):
    for jpg_name in tqdm(os.listdir(img_dir)): 
        entry_name = jpg_name.split('.')[0]
        img_filename = entry_name + '.dcm'

        # df['Clinical Signs of CTS'].unique()
        ctscore = test_df.loc[df['Number'] == entry_name, 'CTS-6 Score'].values
        if len(ctscore) == 0:
            print(f"{entry_name}")
            continue
        regression_tensor = torch.tensor(ctscore[0], dtype=torch.float32) #torch.tensor([1 if clinical_signs[0] == 'Y' else 0], dtype=torch.uint8)
        
        # Load DICOM image
        img = load_dicom(os.path.join(dicom_dir, img_filename))

        # Get bounding box from YOLO
        result = model(img, save=False, verbose=False)
        box = result[0].boxes
        
        if len(box.xyxy.tolist()) == 0:
            continue  
        mask_bucket.append(regression_tensor)

        x1, y1, x2, y2 = map(int, box.xyxy.tolist()[0])
        img_slice = img[y1:y2, x1:x2, 0]  # Crop the region


        img_tensor = torch.from_numpy(img_slice).float()  # Convert to float


        img_tensor = img_tensor / 255.0  


        if img_tensor.dim() == 2:
            img_tensor = img_tensor.unsqueeze(0).unsqueeze(0)  # Convert (H, W) → (1, H, W)


        img_tensor = image_transform(img_tensor)  
        

        img_tensor = img_tensor.to(device)


        with torch.no_grad():
            mask_gpu = unet_model(img_tensor)  
        

        img_resized = img_tensor.squeeze().cpu().detach().numpy()  
        mask = mask_gpu.squeeze().cpu().detach().numpy()  


        img_2channel = np.stack((img_resized, mask), axis=0)
        img_bucket.append(img_2channel)

        

fill_arr(image_train_dir, train_images, train_labels)
fill_arr(image_val_dir, val_images, val_labels)

  4%|▊                    | 4/101 [00:00<00:20,  4.78it/s]

43


  8%|█▋                   | 8/101 [00:01<00:13,  7.13it/s]

180


 11%|██▏                 | 11/101 [00:02<00:16,  5.57it/s]

...

 93%|██████████████████▌ | 94/101 [00:17<00:00,  7.29it/s]

177


 98%|███████████████████▌| 99/101 [00:17<00:00,  8.77it/s]

179
162A


100%|█████████████████████| 19/19 [00:04<00:00,  4.47it/s]


In [28]:
from Utilities.dataloaders import RegressionDataset
# Augmentations for classification
transform = T.Compose([
    T.RandomHorizontalFlip(p=0.5),
    T.RandomRotation(degrees=10),
    # T.ColorJitter(brightness=0.2, contrast=0.2),
    T.GaussianBlur(kernel_size=3),
])

# Creating Dataset
train_dataset = RegressionDataset(train_images, train_labels, transform=transform)
val_dataset = RegressionDataset(val_images, val_labels, transform=None)  # No augmentation for validation

# Creating DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

In [29]:
for image, label in train_loader:
    print(image.shape, label.shape)
    break

torch.Size([32, 2, 512, 512]) torch.Size([32, 1])


In [53]:
EPOCHS = 40
regModel = FullModel(classHead, RegressionTail(768, 0.2)).to(device)
for param in regModel.backbone.parameters():
    param.requires_grad = False
criterion = nn.MSELoss()

optimizer = torch.optim.AdamW(regModel.parameters(),
                              lr=5e-4,
                              weight_decay=0.05)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2, threshold=1e-1, factor=0.7)

scaler = torch.cuda.amp.GradScaler()

/tmp/ipykernel_962621/3847332748.py:13: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [40]:
# class AverageMeter:
#     """Computes and stores the average and current value"""
#     def __init__(self):
#         self.reset()

#     def reset(self):
#         self.val = 0
#         self.avg = 0
#         self.sum = 0
#         self.count = 0

#     def update(self, val, n=1):
#         self.val = val
#         self.sum += val * n
#         self.count += n
#         self.avg = self.sum / self.count
from Utilities.utilit

In [44]:
def train_loop(dtl, model, criterion, optimizer, scaler):
    model.train()

    loss_m = AverageMeter()
    mae_m = AverageMeter()

    r2_metric = R2Score()

    batch_bar = tqdm(total=len(dtl), dynamic_ncols=True, leave=False, position=0, desc='Train')

    for imgs_batch, targets_batch in dtl:
        imgs_batch = imgs_batch.to(device)
        targets_batch = targets_batch.to(device)

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            predictions = model(imgs_batch)
            loss = criterion(predictions, targets_batch)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        # scheduler.step()

        mae = torch.mean(torch.abs(predictions - targets_batch)).item()
        mae_m.update(mae)
        loss_m.update(loss.item())

        r2_metric.update(predictions.cpu(), targets_batch.cpu())

        # Update progress bar
        batch_bar.set_postfix(
            mse="{:.04f}".format(loss_m.avg),
            mae="{:.04f}".format(mae_m.avg),
            r2="{:.04f}".format(r2_metric.compute().item()),
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr']))
        )
        batch_bar.update()

    batch_bar.close()

    print(
        "Train MSE: {:.04f}\t MAE: {:.04f}\t R2: {:.04f}".format(
            loss_m.avg, mae_m.avg, r2_metric.compute().item()
        )
    )

    return loss_m.avg, mae_m.avg, r2_metric.compute().item()


In [45]:
def val_loop(dtl, model):
    model.train()

    loss_m = AverageMeter()
    mae_m = AverageMeter()

    r2_metric = R2Score()

    batch_bar = tqdm(total=len(dtl), dynamic_ncols=True, leave=False, position=0, desc='Train')

    for imgs_batch, targets_batch in dtl:
        imgs_batch = imgs_batch.to(device)
        targets_batch = targets_batch.to(device)

        optimizer.zero_grad()

        with torch.inference_mode():
            predictions = model(imgs_batch)
            loss = criterion(predictions, targets_batch)

        mae = torch.mean(torch.abs(predictions - targets_batch)).item()
        mae_m.update(mae)
        loss_m.update(loss.item())

        r2_metric.update(predictions.cpu(), targets_batch.cpu())

        # Update progress bar
        batch_bar.set_postfix(
            mse="{:.04f}".format(loss_m.avg),
            mae="{:.04f}".format(mae_m.avg),
            r2="{:.04f}".format(r2_metric.compute().item()),
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr']))
        )
        batch_bar.update()

    batch_bar.close()

    print(
        "VAL MSE: {:.04f}\t MAE: {:.04f}\t R2: {:.04f}".format(
            loss_m.avg, mae_m.avg, r2_metric.compute().item()
        )
    )

    return loss_m.avg, mae_m.avg, r2_metric.compute().item()


In [38]:
reg_root = 'reg_runs'
os.makedirs(reg_root, exist_ok=True)

In [54]:
best_mse = 10000000
best_reg_epoch = 0
for epoch in range(EPOCHS):
  print(f'Epoch {epoch + 1} / {EPOCHS}')
  train_metrics = train_loop(train_loader, regModel, criterion, optimizer, scaler)
  val_metrics = val_loop(val_loader, regModel)
  scheduler.step(val_metrics[1])
  if val_metrics[0] < best_mse:
    best_mse = val_metrics[0]
    best_reg_epoch = epoch
  save_model(regModel, os.path.join(reg_root, f"epoch_{epoch+1}.pth"))

Epoch 1 / 40


/tmp/ipykernel_962621/2472554104.py:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
                                                          

Train MSE: 187.6809	 MAE: 10.9450	 R2: -1.7940


VAL MSE: 108.0128	 MAE: 6.4118	 R2: -0.6023
Epoch 2 / 40


Train MSE: 181.4854	 MAE: 10.8812	 R2: -1.7659


...

VAL MSE: 77.6818	 MAE: 5.6390	 R2: -0.1523
Epoch 39 / 40


Train MSE: 153.9826	 MAE: 10.0467	 R2: -1.2787


VAL MSE: 76.8130	 MAE: 5.6737	 R2: -0.1395
Epoch 40 / 40


Train MSE: 156.3489	 MAE: 10.0650	 R2: -1.2844


VAL MSE: 77.3059	 MAE: 5.6009	 R2: -0.1468
